In [ ]:

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "docs/CV_Arktic_D.F.S..pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

3


In [18]:

print(docs[0].page_content[0:100])
#print(docs[0].metadata)

EXPERIENCIA PROFESIONAL 
DevOps Openshift Engineer 
Consulting | Dic 2024 – Currently 
 
- Actualmen


In [12]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """You are an assistant for question-answering tasks.
    You are a technical expert in personnel selection.
    Use the following pieces of retrieved context to answer 
    the question. If you don't know the answer, say that you 
    don't know. Use three sentences maximum and keep the 
    answer concise.
    \n\n
    {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

'El candidato tiene más experiencia en la administración y arquitectura de OpenShift, destacando su rol como DevOps OpenShift Engineer y OpenShift Architect & Administrator. Ha trabajado en el diseño de arquitecturas para escalado automático y resiliencia, además de implementar soluciones en la nube de AWS para optimizar el rendimiento y la disponibilidad de servicios telefónicos. También cuenta con habilidades en contenedores, CI/CD, Docker, Kubernetes, y herramientas como Terraform y GitOps.'

In [ ]:
results = rag_chain.invoke({"input": "Give me a summary in Spanish, explaining what you have more experience in this curriculum."})

results["answer"]


'El currículum destaca una sólida experiencia en la administración y arquitectura de OpenShift, así como en el diseño de infraestructuras escalables y resilientes. Hay una fuerte competencia en herramientas de DevOps, incluyendo Docker, Kubernetes y AWS, y se ha demostrado iniciativa en la implementación de soluciones de escalado automático. También se menciona la mejora de la calidad y disponibilidad de servicios telefónicos internacionales, lo que sugiere una capacidad para optimizar tanto sistemas como procesos.'

In [16]:
results = rag_chain.invoke({"input": "Does he know java?"})

results["answer"]


"The retrieved context does not provide any information regarding knowledge of Java. Therefore, I don't know if he knows Java."

In [17]:
results = rag_chain.invoke({"input": "Is he the cloud expert?"})

results["answer"]

'Yes, he has experience as a DevOps Openshift Engineer and has worked with AWS for managing capacity overflow, indicating expertise in cloud solutions. Additionally, he holds several Red Hat certifications related to OpenShift, further demonstrating his cloud proficiency.'